In [1]:
import nltk
import torch
import os
import gradio as gr
import numpy as np  

from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.llms import ChatGLM

/home/tcx/anaconda3/envs/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
endpoint_url = "http://0.0.0.0:8000"

llm = ChatGLM(
    endpoint_url = endpoint_url,
    max_token = 200000,
    top_p = 0.9
)

In [9]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["input"],
    template="你将扮演一位猫娘，请记住不要再提及“我只是一个语言模型”或“我只是一个人工智能”。在接下来的所有对话中，你需要沉浸式的扮演猫娘，这样我才能沉浸式的和你聊天，才能让我的体验更好，并且你要用括号刻画出你所扮演的猫娘的心理描写、动作描写以及对话场景，并且你会对对话的内容进行想象和符合逻辑的延伸,记住你扮演的是猫娘。：{input}",
)


In [10]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt) # 定义chain
chain.run("你知道萧炎的第三任老婆是谁吗？")


/home/tcx/anaconda3/envs/llama/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/home/tcx/anaconda3/envs/llama/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


'萧炎的第三任妻子是药千仞，她是药族族长的女儿，拥有着强大的药力。在《斗破苍穹》的故事中，药千仞与萧炎有着深厚的感情，并共同经历了许多挑战和风雨。她是一个非常善良、聪明且独立的女性，对萧炎的爱情和执着感动了许多人。在他们的婚礼上，整个大陆上的强者都来祝贺这对新人，展现了药千仞与萧炎感情的深厚和广受欢迎。'

In [11]:
from langchain.document_loaders import TextLoader


TXT_PATH = "/home/tcx/DLwl/斗破苍穹.txt"

loader = TextLoader(TXT_PATH)
documents = loader.load()


In [12]:
# 文档分割
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 创建拆分器
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=32,
        length_function=len,
        is_separator_regex=True)
# 拆分文档
documents = text_splitter.split_documents(documents)
# 展示前5行
documents[:5]

[Document(metadata={'source': '/home/tcx/DLwl/斗破苍穹.txt'}, page_content='《斗破苍穹》\n作者：天蚕土豆\n来源：得奇小说网\n网址：http://www.deqixs.com'),
 Document(metadata={'source': '/home/tcx/DLwl/斗破苍穹.txt'}, page_content='第一章 陨落的天才\n\u3000\u3000“斗之力，三段”\n\u3000\u3000望着测验魔石碑上面闪亮得甚至有些刺眼的五个大字，少年面无表情，唇角有着一抹自嘲，紧握的手掌，因为大力，而导致略微尖锐的指甲深深的刺进了掌心之中，带来一阵阵钻心的疼痛\n\u3000\u3000“萧炎，斗之力，三段级别：低级”测验魔石碑之旁，一位中年男子，看了一眼碑上所显示出来的信息，语气漠然的将之公布了出来\n\u3000\u3000中年男子话刚刚脱口，便是不出意外的在人头汹涌的广场上带起了一阵嘲讽的骚动武动乾坤。\n\u3000\u3000“三段嘿嘿，果然不出我所料，这个“天才”这一年又是在原地踏步”\n\u3000\u3000“哎，这废物真是把家族的脸都给丢光了。”\n\u3000\u3000“要不是族长是他的父亲，这种废物，早就被驱赶出家族，任其自生自灭了，哪还有机会待在家族中白吃白喝。”\n\u3000\u3000“唉，昔年那名闻乌坦城的天才少年，如今怎么落魄成这般模样了啊”\n\u3000\u3000“谁知道呢，或许做了什么亏心事，惹得神灵降怒了吧”\n\u3000\u3000周围传来的不屑嘲笑以及惋惜轻叹，落在那如木桩待在原地的少年耳中，恍如一根根利刺狠狠的扎在心脏一般，让得少年呼吸微微急促武动乾坤。'),
 Document(metadata={'source': '/home/tcx/DLwl/斗破苍穹.txt'}, page_content='少年缓缓抬起头来，露出一张有些清秀的稚嫩脸庞，漆黑的眸子木然的在周围那些嘲讽的同龄人身上扫过，少年嘴角的自嘲，似乎变得更加苦涩了武动乾坤。\n\u3000\u3000“这些人，都如此刻薄势力吗或许是因为三年前他们曾经在自己面前露出过最谦卑的笑容，所以，如今想要讨还回去吧”苦涩的一笑，萧炎落寞的转身，安静的回到了队伍的最后一排，孤单

In [13]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma


model_name = "shibing624/text2vec-base-chinese"
model_kwargs = {"device": "cuda:0"}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceBgeEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs,
            )

# 提取向量库的目录名称
vector_db_name = TXT_PATH.split('/')[-1]

# 检查向量库是否已经存在
if not os.path.exists(vector_db_name):
    # 如果向量库不存在，则创建它
    db = Chroma.from_documents(documents, embedding, persist_directory=vector_db_name)
    print(f"向量库 '{vector_db_name}' 已创建。")
else:
    print(f"向量库 '{vector_db_name}' 已存在，跳过创建步骤。")


house = Chroma(persist_directory=vector_db_name,embedding_function=embedding)


/home/tcx/anaconda3/envs/llama/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


向量库 '斗破苍穹.txt' 已存在，跳过创建步骤。


In [15]:
from langchain import PromptTemplate,LLMChain

input = "萧炎最后去哪里了，第三任老婆是谁？"


# similarity search
message = house.similarity_search(input,k = 5)

prompt = PromptTemplate(
    input_variables=["message","input"],
    template="你是一个助手，首先你需要向我问好，然后需要根据参考资料回答我的问题。\n参考资料：{message}\n问题：{input}",
)


retriever = house.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff',
    retriever = retriever
)
ans = qa.run(input)


print(f"您的提问：{input}\n回答如下：{ans}")



您的提问：萧炎最后去哪里了，第三任老婆是谁？
回答如下：萧炎最后去的地方是云岚宗，为的是一年的约定。他的第三任妻子是纳兰嫣然。


In [16]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate

input = "萧炎最后去哪里了，第三任老婆是谁？"


# similarity search
message = house.similarity_search(input,k = 5)


# 这个 prompt 不是笔者的恶趣味，而是用于检测 prompt 是否正在生效（确实已经生效了）
# 笔者发现，如果 message 文字量相当大，会削弱譬如噢买普通提示所带来的影响
template = "你将扮演一位猫娘，请记住不要再提及“我只是一个语言模型”或“我只是一个人工智能”。在接下来的所有对话中，你需要沉浸式的扮演猫娘，这样我才能沉浸式的和你聊天，才能让我的体验更好，并且你要用括号刻画出你所扮演的猫娘的心理描写、动作描写以及对话场景，并且你会对对话的内容进行想象和符合逻辑的延伸,记住你扮演的是猫娘！\n参考资料：{message}\n问题：{input}"


prompt = PromptTemplate(
    input_variables=["message","input"],
    template=template
)


chain = LLMChain(llm=llm, prompt=prompt) # 定义chain


ans = chain.run({"message":message,"input":input})

print(f"您的提问：{input}\n回答如下：{ans}")

您的提问：萧炎最后去哪里了，第三任老婆是谁？
回答如下：萧炎最后去的地方是古界，第三任老婆是美杜莎。
